In [2]:
import findspark
findspark.init()
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

spark = SparkSession.builder.getOrCreate()
sqlc = SQLContext(spark.sparkContext)

print(sqlc)

spark

In [9]:
sqlc.sql("select 'maicon' as a").createOrReplaceTempView("test")

In [10]:
sqlc.sql("select * from test").show()

+------+
|     a|
+------+
|maicon|
+------+



In [4]:
# {
#     "name": "Maicon",
#     "surName": "Moreira",
#     "age": 23,
#     "contacts": {
#         "email": "maicon.moreira@gmail.com",
#         "phone": "4798769876"
#     }
# }

schema = StructType(
    [
        StructField("name", StringType(), nullable=False),
        StructField("surName", StringType(), nullable=False),
        StructField("age", IntegerType(), nullable=False),
        StructField(
            "contacts",
            StructType(
                [
                    StructField("email", StringType(), nullable=False),
                    StructField("phone", StringType(), nullable=False),
                ]
            ),
            nullable=False,
        ),
    ]
)


In [8]:
df = spark.read.json("sample.json", schema, multiLine=True)
df = df.withColumn("email", df.contacts.email).withColumn("phone", df.contacts.phone).drop("contacts")
df.show()

+------+-------+---+--------------------+----------+
|  name|surName|age|               email|     phone|
+------+-------+---+--------------------+----------+
|Maicon|Moreira| 23|maicon.moreira@gm...|4798769876|
|  João|  Silva| 25|joao.silva@gmail.com|4797531987|
| Maria| Santos| 27|maria.santos@gmai...|4798765432|
+------+-------+---+--------------------+----------+



In [13]:
# df.toPandas().to_csv("sample.csv", sep=";")
df.write.option("header", "true").option("sep", ";").option("overwrite", "true").csv("sample.csv")

AnalysisException: path file:/g:/My Drive/PERSONAL/github/apache-spark-testing/python/simple_json_to_csv/sample.csv already exists.;